In [24]:
import pandas as pd

def get_queries(results_csv):
    with open(results_csv, "r") as file:
        df = pd.read_csv(file)
        return df["query"].tolist()

In [27]:
queries = get_queries("algolia_search_test_results.csv")
print(queries)

['code completion', 'environment variables', 'terminal', 'flutter', 'javafx', 'community edition', 'bookmark', 'uninstall', 'command line', 'wsl', 'code with me', 'code style', 'reformat code', 'run configuration', 'file template', 'file templates', 'notification', 'notifications']


In [34]:
def write_requests(results_csv, column_name):
    with open(results_csv, "r") as file:
        df = pd.read_csv(file)
        print(df[column_name].head(5))
        df[column_name] = queries
        print(df[column_name].head(5))
        # print(df[column_name].values)

        # for query in get_queries("algolia_search_test_results.csv"):

In [35]:
write_requests("algolia_search_test_results.csv", "algolia")

0    auto-completing-code.html\nedit-scala-code.htm...
1    pipenv.html\nbig-data-tools-configuration.html...
2    advanced-settings.html\nterminal-emulator.html...
3    investigate-changes.html\ntest-management-syst...
4    settingsjavafx.html\nnew-project-wizard.html\n...
Name: algolia, dtype: object
0          code completion
1    environment variables
2                 terminal
3                  flutter
4                   javafx
Name: algolia, dtype: object


In [51]:
def algolia_search(queries):
    from algoliasearch.search_client import SearchClient
    import re

    client = SearchClient.create("OHIV241QET", "b40f3aad037ae7bb1b5d8c56431f1bd2")
    index = client.init_index("prod_JETBRAINSCOM_HELP")
    filters = {"filters": "product:idea AND version:2021.3"}

    results = []

    for query in queries:
        res = index.search(query, filters)
        hits = []

        for hit in res["hits"]:
            url = hit["url"]
            page_name = re.search("(?:.+\/)([^#?]+)", url).group(1)
            if page_name not in hits:
                hits.append(page_name)
            if len(hits) >= 5:
                break

        results.append("\n".join(hits))

    return results

In [54]:
def test_search(results_csv, search_type):
    print("Opening the CSV results file:", results_csv)
    with open(results_csv, "r") as file:
        df = pd.read_csv(file)
        queries_from_csv = df["query"].tolist()
        print("Extracted queries. First 5 are:", queries_from_csv[:5])

        if search_type == "algolia":
            print("Performing algolia search and writing results...")
            df[search_type] = algolia_search(queries_from_csv)
        else:
            print("Not a recognized search type")

        print("Saving the CSV")
        df.to_csv(results_csv, index=False)


In [55]:
test_search("algolia_search_test_results.csv", "word_tfidf")

Opening the CSV results file: algolia_search_test_results.csv
Extracted queries. First 5 are: ['code completion', 'environment variables', 'terminal', 'flutter', 'javafx']
Not recognized
Saving the CSV
